In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shutil import move,rmtree
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import os

2023-06-25 22:52:12.207639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-25 22:52:12.310384: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-25 22:52:12.311760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 22:52:13.479438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Creating a directory structure for dataset to use keras API
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/','test/']
labeldirs = ['dogs/','cats/']
for subdir in subdirs:
    for labeldir in labeldirs:
        newdir = dataset_home+subdir+labeldir
        os.makedirs(newdir,exist_ok=True)

In [3]:
# Delete the corrrupt Images from dataset
dataset_dirs = ['PetImages/Dog/','PetImages/Cat/']  # Replace with the path to your dataset directory

corrupted_images = []  # List to store the paths of corrupted images

# Iterate through the dataset directory
for dataset_dir in dataset_dirs:
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Attempt to open the image
                Image.open(file_path)
            except (IOError, OSError):
                # Add the file path to the list of corrupted images
                corrupted_images.append(file_path)

# Print the corrupted images
print("Corrupted Images:")
for img_path in corrupted_images:
    print(img_path)

# Delete the corrupted images
for img_path in corrupted_images:
    os.remove(img_path)

print("Corrupted images removed.")

/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Corrupted Images:
PetImages/Dog/11702.jpg
PetImages/Dog/Thumbs.db
PetImages/Cat/666.jpg
PetImages/Cat/Thumbs.db
Corrupted images removed.


In [4]:
# Move the dataset Images to our created directory structure
testRatio = 0.2 # The ratio of images that will be included in test dataset
np.random.seed(1) #Seeding
src_folders = ['PetImages/Dog/','PetImages/Cat/']
for folder in src_folders:
    for file in os.listdir(folder):
        src = folder+file
        dst_dir = 'train/'
        if np.random.random()<testRatio:
            dst_dir = 'test/'
        dst = dataset_home+dst_dir
        if folder=='PetImages/Cat/':
            dst+='cats/'
        else:
            dst+='dogs/'
        dst+=file
        move(src,dst)